In [ ]:
import socket
from _thread import *


server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

IP_address = socket.gethostname()

Port = 12345

server.bind((IP_address, Port))

server.listen(100)

list_of_clients = []

def clientthread(conn, addr):
    conn.sendto("Welcome".encode(), addr)
    while True:
        message = conn.recv(2048).decode()
        print(message)
        broadcast(message, conn, addr)



def broadcast(message, conn, addr):
	for client in list_of_clients:
		if client[0]!=conn:
			client[0].sendto(f'Message From: {addr}\n{message}'.encode(), client[1])


def remove(connection):
	if connection in list_of_clients:
		list_of_clients.remove(connection)

while True:
    conn, addr = server.accept()
    print("Connected to :", addr)
    list_of_clients.append([conn, addr])
    start_new_thread(clientthread, (conn, addr))


In [ ]:
import socket, _thread

server = socket.socket()

host = socket.gethostname()
port = 12345

server.bind((host, port))

current_turn = 1
totalPlayer = 0
clientList = []

server.listen(5)

def checkValidity(message):
    if message == 'Move' or message == 'Attack' or message == 'Kill' or message == 'capture':
        return True
    return False


colors = ['black', 'white']


def clientThread(conn, addr, num):
    global current_turn
    conn.sendto(f"Welocme Player: {num}\n Your Color is {colors[num-1]}\n".encode(), addr)
    while True:
        message = conn.recv(2048).decode()
        print(message.split()[0])
        if checkValidity(message.split()[0]) and current_turn==num:
            broadcast(message, conn, current_turn)
            if current_turn == 1:
                current_turn = 2
            else:
                current_turn = 1
        elif(current_turn!=num):
            conn.sendto(f"Not Your Turn".encode(), addr)
            print("Wrong Player Trid to Move\n")
        elif not checkValidity(message.split()[0]):
            conn.sendto(f"Invalid Move\n".encode(), addr)
            print("Invalid Move")


def broadcast(message, conn, num):
    for client in clientList:
        if client[0] != conn:
            client[0].sendto(f'Move By Player: {num}\n{message}'.encode(), client[1])

    
while True:
    conn, addr = server.accept()
    print("Connected to :", addr)
    totalPlayer += 1
    if totalPlayer <= 2:
        clientList.append([conn, addr])
        _thread.start_new_thread(clientThread, (conn, addr, totalPlayer))
    else:
        print("Player number exceeded")
        totalPlayer -= 1
        conn.close()


In [1]:
#3
from _thread import *
from threading import Thread


def printFromThread(num):
    print("Hello From Thread: ", num)


threadArr = []
for i in range(50):
    threadArr.append(Thread(target=printFromThread, args=[i+1]))

for i in reversed(threadArr):
    i.start()

Hello From Thread:  50
Hello From Thread:  49
Hello From Thread:  48
Hello From Thread:  47
Hello From Thread:  46
Hello From Thread:  45
Hello From Thread:  44
Hello From Thread:  43
Hello From Thread:  42
Hello From Thread:  41
Hello From Thread:  40
Hello From Thread:  39
Hello From Thread:  38
Hello From Thread:  37
Hello From Thread:  36
Hello From Thread:  35
Hello From Thread:  34
Hello From Thread:  33
Hello From Thread:  32
Hello From Thread:  31
Hello From Thread:  30
Hello From Thread:  29
Hello From Thread:  28
Hello From Thread:  27
Hello From Thread:  26
Hello From Thread:  25
Hello From Thread:  24
Hello From Thread:  23
Hello From Thread:  22
Hello From Thread:  21
Hello From Thread:  20
Hello From Thread:  19
Hello From Thread:  18
Hello From Thread:  17
Hello From Thread:  16
Hello From Thread:  15
Hello From Thread:  14
Hello From Thread:  13
Hello From Thread:  12
Hello From Thread:  11
Hello From Thread:  10
Hello From Thread:  9
Hello From Thread:  8
Hello From Th

In [ ]:
#4
from threading import *
from time import sleep 

limit = int(input("Enter Limit For Producer Arr: "))
items = []
items_cv = Condition()

def producer():
    for i in range(30):
        with items_cv:
            while len(items) > limit:
                items_cv.wait()
            print('Item Produced: ', i)
            items.append(i)
            items_cv.notify()
        sleep(1)

def consumer():
    while True:
        with items_cv:
            while not items:
                items_cv.wait()
            x = items.pop(0)
            items_cv.notify()
        print("Item consumed: ", x)
        sleep(5)

T = Thread(target=consumer)
T.setDaemon(True)
T.start()

producer()

In [3]:
#5
from threading import *
from time import sleep

item_list = []
lock = Lock()

def producer():
    lock.acquire()
    for i in range(10):
        item_list.append(i)
        print("Item Produced: ", i)
        lock.release()
        sleep(2)
        lock.acquire()
    lock.release()

def consumer():
    lock.acquire()
    while item_list:
        print("Item consumed", item_list[0])
        item_list.pop(0)
        lock.release()
        sleep(5)
        lock.acquire()
    
    lock.release()


T1 = Thread(target = producer)
T2 = Thread(target = consumer)
T1.start()
T2.start()
T1.join()
T2.join()

Item Produced:  0
Item consumed 0
Item Produced:  1
Item Produced:  2
Item consumed 1
Item Produced:  3
Item Produced:  4
Item consumed 2
Item Produced:  5
Item Produced:  6
Item Produced:  7
Item consumed 3
Item Produced:  8
Item Produced:  9
Item consumed 4
Item consumed 5
Item consumed 6
Item consumed 7
Item consumed 8
Item consumed 9


In [ ]:
#6
import threading
import time

class Philosopher(threading.Thread):
    running = True 

    def __init__(self, index, forkOnLeft, forkOnRight):
        threading.Thread.__init__(self)
        self.index = index
        self.forkOnLeft = forkOnLeft
        self.forkOnRight = forkOnRight

    def run(self):
        while(self.running):
            time.sleep(3)
            print ('Philosopher %s is hungry.' % self.index+1)
            self.dine()

    def dine(self):
        fork1, fork2 = self.forkOnLeft, self.forkOnRight
        while self.running:
            fork1.acquire()
            locked = fork2.acquire(False) 
            if locked: break 
            fork1.release()
            print ('Philosopher %s swaps forks.' % self.index+1)
            fork1, fork2 = fork2, fork1
        else:
            return
        self.dining()
        fork2.release()
        fork1.release()
 
    def dining(self):			
        print ('Philosopher %s starts eating. '% self.index+1)
        time.sleep(3)
        print ('Philosopher %s finishes eating and leaves to think.' % self.index+1)

def main():
    num = int(input())
    forks = [threading.Semaphore() for n in range(num)] 
    philosophers= [Philosopher(i, forks[i%num], forks[(i+1)%num]) for i in range(num)]

    Philosopher.running = True
    for p in philosophers: p.start()
    time.sleep(30)
    Philosopher.running = False
    print ("Now we're finishing.")
 

if __name__ == "__main__":
    main()


In [ ]:
#7
from _thread import *
import socket
from time import sleep

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()
port = 12345
s.bind((host, port))
s.listen(100)

is_writing = False
content = []
clientList = []
cur_writer = None

def reader(conn, addr):
    for i in content:
        conn.sendto(f'{i[1]}\n'.encode(), addr)

def writer(conn, addr):
    global cur_writer
    while True:
        message = conn.recv(2048).decode().strip()
        print(message)
        if message == 'exit':
            cur_writer = None
            conn.close()
            break
        content.append([f'Writer: {addr}', message])
        print(content)
        broadcast(message)

def broadcast(meassage):
    for i in clientList:
        if i[0] != cur_writer:
            i[0].sendto(f'{meassage}\n'.encode(), i[1])


while True:
    conn, addr = s.accept()
    print(f'Connected to: {addr}')
    conn.sendto('You a reader or writer: '.encode(), addr)
    message = conn.recv(2048).decode().strip()
    print(message == 'reader')
    if message == 'reader':
        start_new_thread(reader, (conn, addr))
        clientList.append([conn, addr])
    elif message == 'writer':
        print('Request for writer: ') 
        if not cur_writer:
            cur_writer = conn
            start_new_thread(writer, (conn, addr))
            clientList.append([conn, addr])
        else:
            conn.sendto('Writer is busy'.encode(), addr)
            sleep(2)
            conn.close()
    print(clientList)